In [22]:
!pip install -U sentence-transformers==2.2.2 transformers==4.30.2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
# !pip install -q gradio==3.50.2 

In [24]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==1.6.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
Using cached scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.5 MB)
ERROR: pip's d

In [25]:

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re
import string as s

from nltk.corpus import stopwords
from sklearn.metrics import make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix, roc_curve, auc

from nltk.corpus import stopwords
from wordcloud import WordCloud

from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.metrics  import f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# import gradio as gr
from sentence_transformers import SentenceTransformer
import joblib
import requests
import re
import html
from bs4 import BeautifulSoup

import numpy as np
import plotly.graph_objects as go

In [6]:
# test_data = pd.read_json("/kaggle/input/bbc-news/bbc_news.json", lines=True)

test_data = test_data[test_data["label_text"] != "entertainment"]
label_dict = {
    "politics": 1,
    "sport": 2,
    "business": 3,
    "tech": 4
}
def mapper_dict(label):
    return label_dict[label]
    
test_data["ID"] = test_data["label_text"].map(mapper_dict)
test_data

,text,label,label_text,ID
1,sydney to host north v south game sydney will ...,2,sport,2
2,uk coal plunges into deeper loss shares in uk ...,1,business,3
3,blair joins school sailing trip the prime mini...,4,politics,1
4,bath faced with tindall ultimatum mike tindall...,2,sport,2
5,banker loses sexism claim a former executive a...,1,business,3
...,...,...,...,...
995,mobile multimedia slow to catch on there is no...,0,tech,4
996,owen determined to stay in madrid england forw...,2,sport,2
997,mobile tv tipped as one to watch scandinavians...,0,tech,4
998,stormy year for property insurers a string of ...,1,business,3


In [28]:
test_data = pd.read_csv("/kaggle/input/dataset-2-news-summarized/summarized_data.csv", index_col=0)
test_data

,text,label,label_text,ID,summarized_text,summarized_len
1,sydney to host north v south game sydney will ...,2,sport,2,Sydney is set to become the host for a norther...,144
2,uk coal plunges into deeper loss shares in uk ...,1,business,3,"UK Coal, the largest coal producer in the UK, ...",147
3,blair joins school sailing trip the prime mini...,4,politics,1,Prime Minister Tony Blair joined school childr...,145
4,bath faced with tindall ultimatum mike tindall...,2,sport,2,"Mike Tindall, represented by agent Dipo Alli, ...",131
5,banker loses sexism claim a former executive a...,1,business,3,"Stephanie Villalba, a former executive at Merr...",140
...,...,...,...,...,...,...
995,mobile multimedia slow to catch on there is no...,0,tech,4,Despite the rapid growth in sales of mobile ph...,160
996,owen determined to stay in madrid england forw...,2,sport,2,"Michael Owen, the England forward, has affirme...",145
997,mobile tv tipped as one to watch scandinavians...,0,tech,4,Mobile TV is gaining traction as a promising i...,150
998,stormy year for property insurers a string of ...,1,business,3,In the face of unprecedented natural disasters...,140


In [29]:
test_x = test_data.text
test_y = test_data.ID

In [30]:
import html
import re

def clean_html_noise(text: str) -> str:
   
    text = html.unescape(text)
    
   
    text = re.sub(r"<.*?>", "", text)
    
  
    text = re.sub(r"\s+", " ", text).strip()
    
    return text


test_x = test_x.apply(clean_html_noise)

In [31]:
def url_remove(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

test_x = test_x.apply(url_remove)


In [32]:

def word_tokenize(text):
    token = re.findall("[\w']+", text)
    return token

test_x = test_x.apply(word_tokenize)

test_x = test_x.apply(lambda x: [word.lower() for word in x])



In [33]:
def remove_stopwords(lst):
    stop_words = set(stopwords.words('english'))
    return [word for word in lst if word.lower() not in stop_words]

test_x = test_x.apply(remove_stopwords)


In [34]:
import string
def remove_punctuations(lst):
    return [''.join(char for char in word if char not in string.punctuation) for word in lst]

test_x = test_x.apply(remove_punctuations)



In [35]:
def remove_number(lst):
    return [''.join(char for char in word if char not in string.digits) for word in lst if word]

test_x = test_x.apply(remove_number)

In [36]:
def remove_extra_words(word_list):
    stopwords = ['href', 'lt', 'gt', 'ii', 'iii', 'ie', 'quot', 'com']
    cleaned_list = []
    for word in word_list:
        if word not in stopwords:
            cleaned_list.append(word)
    return cleaned_list
test_x = test_x.apply(remove_extra_words)


In [37]:

def join_words_with_space(sentence):
    joined_sentence = ' '.join(sentence)
    return joined_sentence

test_x = test_x.apply(join_words_with_space)

In [38]:
BERT_DIR = "/kaggle/input/knn-bert/other/default/1"
knn_bert      = joblib.load(f"{BERT_DIR}/knn_model.pkl")
scaler_bert   = joblib.load(f"{BERT_DIR}/scaler.pkl")
selector_bert = joblib.load(f"{BERT_DIR}/selector.pkl")
bert_model = SentenceTransformer("all-MiniLM-L6-v2")

In [39]:
def classify(text):
    try:
        emb = bert_model.encode([text])
        emb = scaler_bert.transform(emb)
        emb = selector_bert.transform(emb)
        pred = knn_bert.predict(emb)[0]
        return pred
    except Exception as e:
        return f"Error: {str(e)}"

In [41]:
pred = []
for _, row in test_data.iterrows():
    pred.append(classify(row["summarized_text"]))
test_data["prediction"] = pred

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [44]:

def eval_model(y_test,y_pred):
    results = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, average='macro'),
        "recall": recall_score(y_test, y_pred, average='macro'),
        "f1": f1_score(y_test, y_pred, average='macro')
    }

    print("Test set accuracy: {:.4f}".format(results["accuracy"]))
    print("Test set precision: {:.4f}".format(results["precision"]))
    print("Test set recall: {:.4f}".format(results["recall"]))
    print("Test set F1: {:.4f}".format(results["f1"]))
    print("F1 per class:", f1_score(y_test, y_pred, average=None))
    return results


In [45]:
eval_model(test_data["ID"].tolist(), pred)

Test set accuracy: 0.9017
Test set precision: 0.8977
Test set recall: 0.8963
Test set F1: 0.8961
F1 per class: [0.82985075 0.97319588 0.86104784 0.92030848]


{'accuracy': 0.9016990291262136,
 'precision': 0.8976527330251238,
 'recall': 0.8962814153439154,
 'f1': 0.8961007354596734}